In [ ]:
from utils import *
import getpass
from landsatxplore.earthexplorer import EarthExplorer
import tarfile
import boto3

In [ ]:
aws_session = rasterio.session.AWSSession(boto3.Session(), requester_pays=True)

In [ ]:
bbox = read_kml_polygon("data/inputs/landsat/WA.kml")[1]

In [ ]:
user = getpass.getpass("Enter your username?")
password = getpass.getpass("Enter your password?")
ee = EarthExplorer(user, password)

In [ ]:
start_date = "2023-01-01T00:00:00"
end_date: str = "2025-01-01T00:00:00"
platform = "LANDSAT_8"
# query = get_landsat_search_query([67.45, -72.55, 67.55, -72.45], start_date=start_date, end_date=end_date, platform=platform) # Amery ice shelf bed rock
query = get_landsat_search_query(
    [69.0, -68.60, 71.58, -67.60],
    start_date=start_date,
    end_date=end_date,
    platform=platform,
    cloud_cover=50,
)  # Top of Amery ice shelf
# query = get_landsat_search_query(bbox)
features = fetch_landsat_stac_server(query)
feat_dict = find_landsat_scenes_dict(features)

In [ ]:
set([s.split("_")[2] for s in list(feat_dict.keys())])

In [ ]:
r = feat_dict["LC08_L2SR_129109_20241230_20250104_02_T2_SR"]["red"][1]
g = feat_dict["LC08_L2SR_129109_20241230_20250104_02_T2_SR"]["green"][1]
b = feat_dict["LC08_L2SR_129109_20241230_20250104_02_T2_SR"]["blue"][1]

In [ ]:
r_img, rp = stream_scene(r, aws_session)
g_img, gp = stream_scene(g, aws_session)
b_img, bp = stream_scene(b, aws_session)

In [ ]:
rgb = apply_gamma(cv.merge([r_img, g_img, b_img]), 1.0, True)
plt.imshow(rgb)

In [ ]:
path_num = "128109"
required_keys = [k for k in list(feat_dict.keys()) if path_num in k]
required_scenes = [feat_dict[k]["scene_id"] for k in required_keys]
len(required_scenes)

In [ ]:
for scene in required_scenes:
    output_dir = f"data/inputs/{platform}_{path_num}"
    os.makedirs(f"data/inputs/{platform}_{path_num}", exist_ok=True)
    print(f"Downloading {scene}")
    ee.download(scene, output_dir)

In [ ]:
tar_files = [
    f for f in glob.glob(f"data/inputs/{platform}_{path_num}/**") if f.endswith(".tar")
]
for tar_file in tar_files:
    extraction_dir = os.path.join(
        output_dir, os.path.splitext(os.path.basename(tar_file))[0]
    )
    os.makedirs(extraction_dir, exist_ok=True)
    with tarfile.open(tar_file) as tf:
        tf.extractall(extraction_dir)

In [ ]:
red = f"data/inputs/{platform}_{path_num}/LC08_L1GT_127111_20141103_20201016_02_T2/LC08_L1GT_127111_20141103_20201016_02_T2_B4.TIF"
green = f"data/inputs/{platform}_{path_num}/LC08_L1GT_127111_20141103_20201016_02_T2/LC08_L1GT_127111_20141103_20201016_02_T2_B3.TIF"
blue = f"data/inputs/{platform}_{path_num}/LC08_L1GT_127111_20141103_20201016_02_T2/LC08_L1GT_127111_20141103_20201016_02_T2_B2.TIF"

redf = flip_img(rasterio.open(red).read())[:, :, 0]
redf = apply_gamma(redf, 1.0, True)

greenf = flip_img(rasterio.open(green).read())[:, :, 0]
greenf = apply_gamma(greenf, 1.0, True)

bluef = flip_img(rasterio.open(blue).read())[:, :, 0]
bluef = apply_gamma(bluef, 1.0, True)


img = cv.merge([redf, bluef, greenf])
img.shape

In [ ]:
plt.imshow(img)

In [ ]:
true_color_dir = f"data/inputs/{platform}_{path_num}/true_color"
os.makedirs(true_color_dir, exist_ok=True)
scenes_dirs = [
    dir
    for dir in glob.glob(f"data/inputs/{platform}_{path_num}/**")
    if dir.endswith("_T2")
]
for dir in scenes_dirs:
    print(f"Now processing {os.path.basename(dir)}.")
    files = glob.glob(f"{dir}/**")
    r_band = list(filter(lambda f: f.endswith("_B4.TIF"), files))[0]
    g_band = list(filter(lambda f: f.endswith("_B3.TIF"), files))[0]
    b_band = list(filter(lambda f: f.endswith("_B2.TIF"), files))[0]
    true_bands = [r_band, g_band, b_band]
    make_landsat_true_color_scene(
        true_bands, f"{os.path.join(true_color_dir, os.path.basename(dir))}_TC.TIF"
    )

In [ ]:
true_color_ds_dir = f"data/inputs/{platform}_{path_num}/true_color_ds"
os.makedirs(true_color_ds_dir, exist_ok=True)
tc_files = glob.glob(f"data/inputs/{platform}_{path_num}/true_color/**")
for tcf in tc_files:
    downsample_dataset(tcf, 0.2, os.path.join(true_color_ds_dir, os.path.basename(tcf)))

In [ ]:
show(
    rasterio.open(
        f"data/inputs/{platform}_{path_num}/true_color_ds/LC08_L1GT_127111_20150106_20201016_02_T2_TC.TIF"
    ).read()
)